In [1]:
import ecole

In [2]:
default_scip_params = {'separating/maxrounds': 0,
                       'separating/maxroundsroot': 0,
                       'separating/maxcuts': 0,
                       'separating/maxcutsroot': 0,
                       'presolving/maxrounds': 0,
                       'presolving/maxrestarts': 0,
                       'propagating/maxrounds':0,
                       'propagating/maxroundsroot':0,
                       'lp/initalgorithm':'d',
                       'lp/resolvealgorithm':'d',
                       'limits/time': 3600}
class EcoleBranching(ecole.environment.Branching):
    def __init__(
        self,
        observation_function='default',
        information_function='default',
        reward_function='default',
        scip_params='default',
        pseudo_candidates=False,
    ):
        '''
        Args:
            scip_params (dict, str): Custom dictionary or one of: 'default', 
                'ml4co_item_placement', 'ml4co_load_balancing', 'ml4co_anonymous'.
        '''
        # save string names so easy to initialise new environments
        if type(observation_function) == str:
            self.str_observation_function = observation_function
        else:
            self.str_observation_function = None
        if type(information_function) == str:
            self.str_information_function = information_function
        else:
            self.str_information_function = None
        if type(reward_function) == str:
            self.str_reward_function = reward_function
        else:
            self.str_reward_function = None
        if type(scip_params) == str:
            self.str_scip_params = scip_params
        else:
            self.str_scip_params = None

        # init reward functions from strings if needed
        if reward_function == 'default':
            reward_function = ({
                     'num_nodes': -ecole.reward.NNodes(),
                     'lp_iterations': -ecole.reward.LpIterations(),
                     'primal_integral': -ecole.reward.PrimalIntegral(),
                     'dual_integral': ecole.reward.DualIntegral(),
                     'primal_dual_integral': -ecole.reward.PrimalDualIntegral(),
                     'solving_time': -ecole.reward.SolvingTime(),
                 })
        if information_function == 'default':
            information_function=({
                     'num_nodes': ecole.reward.NNodes().cumsum(),
                     'lp_iterations': ecole.reward.LpIterations().cumsum(),
                     'solving_time': ecole.reward.SolvingTime().cumsum(),
                 })
        if observation_function == 'default':    
            observation_function=(
                     ecole.observation.NodeBipartite()
                 )
        if scip_params == 'default':
            scip_params = default_scip_params
        
        super(EcoleBranching, self).__init__(
            observation_function=observation_function,
            information_function=information_function,
            reward_function=reward_function,
            scip_params=scip_params,
            pseudo_candidates=False,
        )

In [22]:
class StrongBranchingAgent:
    def __init__(self, pseudo_candidates=False, name='sb'):
        self.name = name
        self.strong_branching_function = ecole.observation.StrongBranchingScores(pseudo_candidates=False)

    def before_reset(self, model):
        self.strong_branching_function.before_reset(model)
    
    def extract(self, model, done, *args):
        return self.strong_branching_function.extract(model, done)

    def action_select(self, action_set, model, done):
        scores = self.extract(model, done)[action_set]
        action = scores.argmax()
        
#         print('method 2')
#         scores = self.extract(model, done)
#         action = scores.argmax()
#         best_action = action_set[scores.argmax()]
#         print(f'action set: {action_set}')
#         print(f'raw scores: {scores}')
#         print(scores.shape, scores, best_action)
        
        return action, scores
    
    
agent = StrongBranchingAgent()
env = EcoleBranching(observation_function='default',
                          information_function='default',
                          reward_function='default',
                          scip_params='default')
env.seed(0)
instances = ecole.instance.SetCoverGenerator(n_rows=100, n_cols=100, density=0.05)

In [23]:
num_episodes = 1
for ep in range(num_episodes):
    # find an instance not pre-solved by environment
    obs = None
    while obs is None:
        env.seed(0)
        instance = next(instances)
        instance_before_reset = instance.copy_orig()
        agent.before_reset(instance)
        obs, action_set, reward, done, info = env.reset(instance)
    
    # solve instances
    t = 1
    while not done:
        m = env.model.as_pyscipopt()
        prev_dual = m.getDualbound()
        action_idx, scores = agent.action_select(action_set, env.model, done)
        action = action_set[action_idx]
        obs, action_set, reward, done, info = env.step(action)
        m = env.model.as_pyscipopt()
        curr_dual = m.getDualbound()
        print(f'Step {t} action {action} | SB score: {scores[action_idx]} | Change in dual bound: {curr_dual-prev_dual}')
        t += 1

method 2
action set: [ 1  3  4 11 17 19 20 22 27 28 29 30 31 32 33 34 38 47 48 49 51 54 55 59
 67 70 72 77 78 84 93 94 97 99]
raw scores: [         nan   4.51171875          nan  49.82087054   6.23828125
          nan          nan          nan          nan          nan
          nan  99.6501625           nan          nan          nan
          nan          nan  17.70598958          nan  27.8125
   3.75841346          nan  36.43446181          nan          nan
          nan          nan  19.04765625  99.6501625    2.98307292
  21.41015625  61.58723958 112.44140625  27.8125      19.72265625
          nan          nan          nan  70.09765625          nan
          nan          nan          nan          nan          nan
          nan          nan  56.62566551  86.03203125  22.99609375
          nan  27.64696181          nan          nan  86.03203125
  85.61328125          nan          nan          nan  82.85807292
          nan          nan          nan          nan          nan
        